In [ ]:
import requests
import json
url = ('https://newsapi.org/v2/everything?'
       #'category=general&'
       'domains=bbc.co.uk&'
       #'country=gb&'
       'pageSize=99&'
       'page=1&'
       'from=2019-05-05&to=2019-06-03&'
       'apiKey=34e73062364d41b180a6f6a9625bff79')
response = requests.get(url)

with open('data/data.json', 'w') as outfile:
    json.dump(response.json(), outfile)

In [ ]:
import pandas as pd

news_df = pd.DataFrame(columns=['Title', 'Date', 'Stems'])

In [ ]:

with open("data/data.json") as f:
        loaded_json = json.load(f)
        for i in range(len(loaded_json["articles"])):
            if loaded_json["articles"][i]["content"] is not None:
                news_df = news_df.append({'Title': loaded_json["articles"][i]["title"], 'Date': loaded_json["articles"][i]["publishedAt"]}, ignore_index=True)
                #print(loaded_json["articles"][i]["title"])
                

In [ ]:
news_df

## Stemming

In [ ]:
# import spacy
# from spacy import displacy
# from collections import Counter
# import en_core_web_sm
# nlp = en_core_web_sm.load()

In [ ]:
import nltk
from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
porter = PorterStemmer()

from nltk.tokenize import sent_tokenize, word_tokenize
def stemSentence(sentence):
    token_words=tokenizer.tokenize(sentence)
    token_words
    stop_words = set(stopwords.words('english'))
    #print(stop_words)
    filter_sentense = [w for w in token_words if not w in stop_words]
    
    stem_sentence=[]
    for word in filter_sentense:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [ ]:
sentence=news_df.Title[0]
x=stemSentence(sentence)
print(sentence)
print(x)


In [ ]:
for i in news_df.index:
    news_df.Stems[i] = stemSentence(news_df.Title[i])

In [ ]:
wordfreq = []
master_wordlist = []
for i in news_df.index:
    wordlist = news_df.Stems[i].split()

    master_wordlist+= wordlist
#print(master_wordlist)
# print(set(master_wordlist))
# for w in master_wordlist:
#     wordfreq.append(master_wordlist.count(w))

# ww = list(zip(master_wordlist, wordfreq))
# print("Pairs\n" + str(ww[:]))

In [ ]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(zip(wordlist,wordfreq))

In [ ]:
word_freq = wordListToFreqDict(master_wordlist)
# print(word_freq)

In [ ]:
# for x, y in word_freq.items():
#     print(x, y)
# print(word_freq.values())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.io import output_notebook

output_notebook()

In [ ]:
fruits = list(word_freq.keys())
counts = list(word_freq.values())

# sorting the bars means sorting the range factors
sorted_fruits = sorted(fruits, key=lambda x: counts[fruits.index(x)])

p = figure(x_range=sorted_fruits, plot_height=500, plot_width=1400, title="Word frequencies",
           toolbar_location="below")

p.vbar(x=fruits, top=counts, width=0.9)
p.xaxis.major_label_orientation = "vertical"
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_font_size = "25pt"
p.xaxis.major_label_text_font_size = "10pt"
p.y_range.start = 0

show(p)

## Google trends (hourly data)

In [ ]:
import pandas as pd
from pytrends.request import TrendReq
import os  
import numpy as np
pytrend = TrendReq(hl='en-US', tz=360)

In [ ]:
#functions
def get_interest_and_region(kw_list,pytrend):
    if not (os.path.isfile('Data/'+str(kw_list[0])+'.pkl') and os.path.isfile('Data/'+str(kw_list[0])+'_regions'+'.pkl')):
        print('Term not downloaded yet, downloading now..')
        
        #pytrend = TrendReq(hl='en-US', tz=360, timeout=(10,25), proxies=['https://34.203.233.13:80',], retries=5, backoff_factor=1)

        search_df = pytrend.get_historical_interest(kw_list, year_start=2005, year_end=2018).reset_index()

        search_df.to_pickle('Data/'+str(kw_list[0])+'.pkl')
        region_df = pytrend.interest_by_region(resolution='COUNTRY', inc_low_vol=False, inc_geo_code=False)
        region_df.to_pickle('Data/'+str(kw_list[0])+'_regions'+'.pkl')
        print('term with regional data saved')
        return(search_df, region_df)
    else:
        print('term has been found..')
        pytrend = TrendReq(hl='en-US', tz=360)   
        term_df = pd.read_pickle('Data/'+str(kw_list[0])+'.pkl')
        region_df = pd.read_pickle('Data/'+str(kw_list[0])+'_regions'+'.pkl')
        print(term_df.head())
        return(term_df, region_df)
        
        
def plot_interest_in_time(df2, kw_list):

    from bokeh.io import show, output_file
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from datetime import datetime as dt
    from bokeh.models import DatetimeTickFormatter

    output_notebook()
    TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
    TOOLTIPS = [
        ('month', '@x{datetime}'),
        ('Total searches', '@y'),
    ]

    p = figure(title="Monthly-wise total search of term - "+str(kw_list[0]), y_axis_type="linear", plot_height = 400,
               tools = TOOLS, tooltips = TOOLTIPS, plot_width = 800)


    p.xaxis.axis_label = 'Month'
    p.yaxis.axis_label = 'Total search'
    #p.circle(2010, temp_df.IncidntNum.min(), size = 10, color = 'red')

    p.line(df2.index, df2[str(kw_list[0])],line_color="purple", line_width = 3)

    p.xaxis.formatter=DatetimeTickFormatter(

            days=["%d %B %Y"],
            months=["%d %B %Y"],
            years=["%d %B %Y"],
        )
    # output_file("line_chart.html", title="Line Chart")
    show(p)


def interest_test(kw_list, pytrend):
    #create patload
    pytrend.build_payload(kw_list, timeframe='all')
    interest_over_time_df = pytrend.interest_over_time()
    search_df.to_pickle('Data/trends/'+str(kw_list[0])+'.pkl')
    return(interest_over_time_df)
    #print(interest_over_time_df)
#edef

In [ ]:
kw_list = ['greece crisis']

#time_df, regions_df = get_interest_and_region(kw_list, pytrend)

df = interest_test(kw_list,pytrend)
plot_interest_in_time(df, kw_list)


In [ ]:
## Downsampling to monthly
df2 = time_df.resample('M', on='date').mean()
print(df2.index)

In [ ]:
plot_interest_in_time(df2,kw_list)

In [ ]:
# #add year and month
# term_csv['Year'] = term_csv.date.apply(lambda x: x.year)
# term_csv['Month'] = term_csv.date.apply(lambda x: x.month)
# term_csv.head()

In [ ]:
# temp_df = term_csv.groupby(['Year']).mean().reset_index()

# temp_df

In [ ]:
region_df = pd.read_pickle('Data/crisis_regions'+'.pkl')

In [ ]:
region_df.index[0]

# Google trends (Interest over time)

In [ ]:
import pandas as pd
from pytrends.request import TrendReq
import os  
import numpy as np
pytrend = TrendReq(hl='en-US', tz=360)

In [ ]:
def interest_test(kw_list, pytrend):
    #create patload
    pytrend.build_payload(kw_list, timeframe='all')
    interest_over_time_df = pytrend.interest_over_time()
    interest_over_time_df.to_pickle('Data/trends/'+str(kw_list[0].replace(" ", "_"))+'_'+str(kw_list[1].replace(" ", "_")) +'.pkl')
    region_df = pytrend.interest_by_region(resolution='COUNTRY', inc_low_vol=False, inc_geo_code=False)
    #region_df.to_pickle('Data/trends/'+str(kw_list[0].replace(" ", "_"))+'_regions'+'.pkl')
    #return(interest_over_time_df)
    #print(interest_over_time_df)
#edef
def plot_interest_in_time(df2, kw_list):

    from bokeh.io import show, output_file
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from datetime import datetime as dt
    from bokeh.models import DatetimeTickFormatter

    output_notebook()
    TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
    TOOLTIPS = [
        ('month', '@x{datetime}'),
        ('Total searches', '@y'),
    ]

    p = figure(title="Monthly-wise total search of term - "+str(kw_list[0]), y_axis_type="linear", plot_height = 400,
               tools = TOOLS, tooltips = TOOLTIPS, plot_width = 800)


    p.xaxis.axis_label = 'Month'
    p.yaxis.axis_label = 'Total search'
    #p.circle(2010, temp_df.IncidntNum.min(), size = 10, color = 'red')

    p.line(df2.index, df2[str(kw_list[0])],line_color="purple", line_width = 3)

    p.xaxis.formatter=DatetimeTickFormatter(

            days=["%d %B %Y"],
            months=["%d %B %Y"],
            years=["%d %B %Y"],
        )
    # output_file("line_chart.html", title="Line Chart")
    show(p)
#edef

#### Functions to cache countries with tags

In [ ]:
tags = [ 'humanitarian crisis',
         'natural disaster',
         'environmental crisis',
         'disability',
         'gender',
         'genital mutilation',
         'racism',
         'genocide',
         'civil war',
         'terrorism',
         'infectious disease',
         'political revolution',
         'political prisoner',
         'amnesty',
         'corruption',
         'health',
         'gender inequality',
         'rape',
         'ebola',
         'aids',
         'hiv',
         'technology',
         'artificial intelligence',
         'military',
         'war',
         'climate change',
         'starvation',
         'food shortage',
         'dehydration',
         'water shortage',
         'attack',
         'aggression',
         'logistics',
         'nutrition',
         'protection',
         'shelter',
         'drinking water',
         'sanitation',
         'hygiene',
         'refugee camp',
         'education',
         'emergency communication system',
         'food security',
         'human rights',
         'children',
         'pregnancy',
         'old age',
         'justice',
         'homelessness',
         'art',
         'indigenous peoples'
       ]

In [ ]:
countries = [
    'Portugal',
    'Russia',
    'Myanmar',
    'Italy',
    'Ireland'
]

In [ ]:
# Download country and terms
region_df = pd.read_pickle('Data/crisis_regions'+'.pkl')
for i in range (len(countries)):
    for j in range (len(countries)-1):
        for term in tags:
            #kw_list = [country+' '+term, country+1]
    #        interest_test(kw_list, pytrend)
            if not (countries[i] == countries[j]):
                kw_list = [countries[i] + ' ' +term, countries[j] +' ' +term]
                interest_test(kw_list, pytrend)
                #print(kw_list)

In [ ]:
# # plot an example
# ex_df = pd.read_pickle('Data/trends/Algeria_genocide.pkl')
# print(ex_df)
# kw_li = ['Algeria genocide']
# plot_interest_in_time(ex_df, kw_li)


In [ ]:
# # testing comnparison of terms
# 
# interest_test(comp_list, pytrend)

In [ ]:
#ploting
ex_df = pd.read_pickle('Data/trends/Algeria_aids_Greece_aids.pkl')
comp_list = ['Algeria aids', 'Greece aids']

In [ ]:
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.io import output_notebook
from datetime import datetime as dt
from bokeh.models import DatetimeTickFormatter

output_notebook()
TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
TOOLTIPS = [
    ('month', '@x{datetime}'),
    ('Total searches', '@y'),
]

p = figure(title="Monthly-wise total search of term - "+str(comp_list), y_axis_type="linear", plot_height = 400,
               tools = TOOLS, tooltips = TOOLTIPS, plot_width = 800)


p.xaxis.axis_label = 'Month'
p.yaxis.axis_label = 'Total search'
#p.circle(2010, temp_df.IncidntNum.min(), size = 10, color = 'red')

p.line(ex_df.index, ex_df[str(comp_list[0])],line_color="purple", line_width = 3)
p.line(ex_df.index, ex_df[str(comp_list[1])],line_color="purple", line_width = 3)

p.xaxis.formatter=DatetimeTickFormatter(

        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )
# output_file("line_chart.html", title="Line Chart")
show(p)

## Functions to get the results from the caches files
### These functions let you get the occurences for a given country (cached only) by specifying 
### either a particular date or a month

In [ ]:
import pandas as pd
from pytrends.request import TrendReq
import os  
import numpy as np
import datetime as dt
import glob
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def get_cause_by_date(country, timestamp):
#theloume max to mhna    
    time_max = []
    timestamp_df = pd.DataFrame(columns=('timestamp', 'occurences', 'reason'))
    
    for name in glob.glob('Data/trends/'+country+'*'):
        country_term = pd.read_pickle(name)
        if not country_term.empty:
            #print(name)
            
            spike_cols = [col for col in country_term.columns if country in col]
            cc = country_term.loc[timestamp][spike_cols]
            #print(cc[0])
            timestamp_df = timestamp_df.append({'timestamp': timestamp, 'occurences': cc[0], 'reason': spike_cols[0]}, ignore_index=True)
            #print(str(spike_cols[0].split()[1]))
    return(timestamp_df.max())
#edef

def get_cause_by_month(country, timestamp):
    timestamp2_df = pd.DataFrame(columns=('timestamp', 'occurences', 'reason'))
    for name in glob.glob('Data/trends/'+country+'*'):
        country_term = pd.read_pickle(name)
        if not country_term.empty:
            df = country_term.groupby(pd.TimeGrouper('M')).max()
            #print(df.loc[timestamp])
            spike_cols = [col for col in country_term.columns if country in col]
            #df = country_term.groupby(pd.Grouper(key='date', freq='60s'))
            cc = country_term.loc[timestamp][spike_cols]
            #print(cc.iloc[0])
            timestamp2_df = timestamp2_df.append({'timestamp': timestamp, 'occurences': cc.iloc[0][0], 'reason': spike_cols[0]}, ignore_index=True)
    #print('the max index is: '+str(timestamp2_df.index.mac()))
    return(timestamp2_df.ix[timestamp2_df.occurences.idxmax()])

def get_cause_by_month_with_window(country, timestamp, window):
    timestamp3_df = pd.DataFrame(columns=('timestamp', 'occurences', 'reason'))
    for name in glob.glob('Data/trends/'+country+'*'):
        country_term2 = pd.read_pickle(name)
        if not country_term2.empty:
            #this dataframe groups by month
            df2 = country_term2.groupby(pd.TimeGrouper('M')).max()
            #print(df2)
            
            #find the indexes of the date index(column) that refers to the monthly timestamp
            
            #get column name that matches the country name (in order to avoid the reference country)
            spike_cols2 = [col for col in country_term2.columns if country in col]
            
            cc2 = country_term2.loc[timestamp][spike_cols2]
            #print(cc2)
            timestamp3_df = timestamp3_df.append({'timestamp': timestamp, 'occurences': cc2.iloc[0][0].astype('float'), 'reason': spike_cols2[0]}, ignore_index=True)
            #print(timestamp3_df)
    #timestamp3_df.occurences = timestamp3_df.occurences.astype('float64')
    #max_index = timestamp3_df.occurences.idxmax()
    #print(timestamp3_df['occurences'].max())
    
    max_index = (timestamp3_df.ix[timestamp3_df.occurences.idxmax()].name)
    max_value  = timestamp3_df.ix[timestamp3_df.occurences.idxmax()].occurences
    #print(timestamp3_df.ix[timestamp3_df.occurences.idxmax()])
    if not (max_index-window < 0):
        
        #go back to all the dataframes and find the one that matches the max and return the time window on that
        
        max_reason = timestamp3_df.iloc[max_index].reason
        max_reason2 = max_reason.replace(" ", "_")
        #print(max_reason2)
        for name in glob.glob('Data/trends/'+max_reason2+'*'):
            country_term3 = pd.read_pickle(name)
            #print(country_term3)
            if not country_term3.empty:
                #print(country_term3)
                my_old_index = country_term3.loc[(country_term3[max_reason] == max_value) & (country_term3.index == timestamp)]
                
                
                
                #my_index = (country_term3.loc[country_term3[max_reason] == max_value && country_term3.index.match(timestamp)].index)
                if not my_old_index.empty:
                    #print(my_old_index.index.get_indexer)
                    my_new_index = (country_term3.index.get_loc(my_old_index.index.values[0]))
                    #print(my_new_index)
                    return(country_term3.iloc[my_new_index - window: my_new_index+window])
                    #window_stop_row = country_term3[country_term3.index < timestamp].iloc[0]
                    #return(window_stop_row)

                
    
    #return(timestamp3_df.iloc[(max_index-window):(max_index+window)])
           

In [ ]:
#print(get_cause_by_date('Egypt', '2014-01-01'))
#print(get_cause_by_month('Greece', '2008-07'))
print(get_cause_by_month_with_window('France', '2017-12', 2))

In [ ]:
## Fix the reason 
### when asked return the occurences around a window

In [1]:
from TWB import trends

In [2]:
print(trends.Trends('France', '2017-12', 2).plot_general_information_for_a_country)

Loading BokehJS ...

None


In [2]:
print(trends.Trends('France', '2017-12', 2).download_trends_for_a_country_general)

Data/trends/countries/France.pkl
You are lucky, country already downloaded
None


In [3]:
print(trends.Trends('France', '2017-12', 2).get_maximum_occurences_on_country_by_month_with_window)

            France political revolution  Italy political revolution isPartial
date                                                                         
2017-10-01                           45                           4     False
2017-11-01                           33                           4     False
2017-12-01                           30                           3     False
2018-01-01                           21                           1     False


In [ ]:
#asdjhas